# RF mapping amination: a stand-alone notebook

Tony Fu, July 2022

## Imports

In [1]:
import copy

import numpy as np
import torch
from torchvision import models
from torchvision.models import AlexNet_Weights, VGG16_Weights
import matplotlib.pyplot as plt
%matplotlib ipympl

## Specify the unit (i.e., convolutional kernel) of interest:

**NOTE**: conv_i = 0 means conv1.

In [ ]:
model = models.alexnet(weights=AlexNet_Weights.IMAGENET1K_V1)
model_name = 'alexnet'
# model = models.vgg16(weights=VGG16_Weights.IMAGENET1K_V1)
# model_name = 'vgg16'

conv_i = 0
unit_i = 11

In [ ]:
i_conv1 =  0   # Store layer indices ("i_...") for particular layers
i_conv2 =  3
i_conv3 =  6
i_conv4 =  8
i_conv5 = 10

i_conv1  =  0   # Store layer indices ("i_...") for particular layers
i_conv2  =  2
i_conv3  =  5
i_conv4  =  7
i_conv5  = 10
i_conv6  = 12
i_conv7  = 14
i_conv8  = 17
i_conv9  = 19
i_conv10 = 21
i_conv11 = 24
i_conv12 = 26
i_conv13 = 28

## Specify the mapping parameters

## Helper functions and classes

In [ ]:
class HookFunctionBase:
    """
    A base class that register a hook function to all specified layer types
    (excluding all container types) in a given model. The child class must
    implement hook_function(). The child class must also call
    self.register_forward_hook_to_layers() by itself.
    """
    def __init__(self, model, layer_types):
        """
        Constructs a HookFunctionBase object.

        Parameters
        ----------
        model : torchvision.models
            The neural network.
        layer_types : tuple of torch.nn.Modules
            A tuple of the layer types you would like to register the forward
            hook to. For example, layer_types = (nn.Conv2d, nn.ReLU) means
            that all the Conv2d and ReLU layers will be registered with the
            forward hook.
        """
        self.model = copy.deepcopy(model)
        self.layer_types = layer_types

    def hook_function(self, module, ten_in, ten_out):
        raise NotImplementedError("Child class of HookFunctionBase must "
                                  "implement hookfunction(self, module, ten_in, ten_out)")

    def register_forward_hook_to_layers(self, layer):
        # If "model" is a leave node and matches the layer_type, register hook.
        if (len(list(layer.children())) == 0):
            if (isinstance(layer, self.layer_types)):
                layer.register_forward_hook(self.hook_function)

        # Otherwise (i.e.,the layer is a container type layer), recurse.
        else:
            for i, sublayer in enumerate(layer.children()):
                self.register_forward_hook_to_layers(sublayer)